In [37]:
from importlib import import_module
import os
import logging
from datetime import datetime

from dask.distributed import Client, LocalCluster

from madmex.management.base import AntaresBaseCommand

from madmex.indexing import add_product_from_yaml, add_dataset, metadict_from_netcdf
from madmex.util import yaml_to_dict, mid_date, parser_extra_args
from madmex.recipes import RECIPES
from madmex.wrappers import gwf_query
from madmex.settings import INGESTION_PATH
from madmex.util import join_dicts

In [38]:
path = os.path.join(INGESTION_PATH, 'recipes', "l7_oax_recipe_0405_test_pl_2")
recipe_meta = RECIPES["landsat_madmex_002"]
product = recipe_meta['product']
fun = recipe_meta['fun']
yaml_file = recipe_meta['config_file']
begin = datetime.strptime("2004-01-01", '%Y-%m-%d')
end = datetime.strptime("2005-12-31", '%Y-%m-%d')
center_dt = mid_date(begin, end)
time = (begin, end)

In [39]:
gwf_kwargs = {'lat': None, 'long': None, 'region': 'Oaxaca', 'begin': '2004-01-01', 'end': '2005-12-31', 'resolution': None, 'tilesize': None, 'origin': None, 'proj4': None}

In [40]:
dict_list = []
for prod in product:
    gwf_kwargs.update(product = prod)
    try:
        dict_list.append(gwf_query(**gwf_kwargs, view=False))
        # Exception is in case one of the product hasn't been registered in the datacube
    except Exception as e:
        pass
iterable = join_dicts(*dict_list).items()

In [53]:
scheduler_file= '/shared_volume/scheduler.json'
client = Client(scheduler_file=scheduler_file)
client.restart()
C = client.map(fun, iterable, pure=False, **{'center_dt': center_dt, 'path': path})
nc_list = client.gather(C)
n_tiles = len([x for x in nc_list if x is not None])

In [54]:
nc_list

['/shared_volume/datacube/datacube_ingest/recipes/l7_oax_recipe_0405_test_pl_2/madmex_002_40_-40_2004-12-31.nc',
 '/shared_volume/datacube/datacube_ingest/recipes/l7_oax_recipe_0405_test_pl_2/madmex_002_42_-34_2004-12-31.nc',
 '/shared_volume/datacube/datacube_ingest/recipes/l7_oax_recipe_0405_test_pl_2/madmex_002_38_-35_2004-12-31.nc',
 '/shared_volume/datacube/datacube_ingest/recipes/l7_oax_recipe_0405_test_pl_2/madmex_002_45_-39_2004-12-31.nc',
 '/shared_volume/datacube/datacube_ingest/recipes/l7_oax_recipe_0405_test_pl_2/madmex_002_39_-40_2004-12-31.nc',
 '/shared_volume/datacube/datacube_ingest/recipes/l7_oax_recipe_0405_test_pl_2/madmex_002_41_-36_2004-12-31.nc',
 '/shared_volume/datacube/datacube_ingest/recipes/l7_oax_recipe_0405_test_pl_2/madmex_002_44_-40_2004-12-31.nc',
 '/shared_volume/datacube/datacube_ingest/recipes/l7_oax_recipe_0405_test_pl_2/madmex_002_46_-37_2004-12-31.nc',
 '/shared_volume/datacube/datacube_ingest/recipes/l7_oax_recipe_0405_test_pl_2/madmex_002_37_-39

In [48]:
product_description = yaml_to_dict(yaml_file)

In [55]:
def write_and_index(nc):
            try:
                print("Adding %s to datacube database" % nc)
                pr, dt = add_product_from_yaml(yaml_file, "l7_oax_recipe_0405_test_pl_2")
                metadict = metadict_from_netcdf(file=nc, description=product_description,
                                                center_dt=center_dt, from_dt=begin,
                                                to_dt=end, algorithm=options['recipe'])
                add_dataset(pr=pr, dt=dt, metadict=metadict, file=nc)
                return True
            except:
                print ('Error in nc = ', nc)
                return False

In [61]:
client.restart()
client = Client(scheduler_file=scheduler_file)
C = client.map(write_and_index, nc_list)
r = client.gather(C)


In [62]:
client.restart()

Client Scheduler: tcp://100.96.4.6:8786 Dashboard: http://100.96.4.6:8787/status,Cluster Workers: 12 Cores: 12 Memory: 156.00 GB
